### 1. create charts with different Color; creating a new excel file "new.xlsx"

In [1]:
from openpyxl import load_workbook
from openpyxl.chart import BarChart, Series, Reference
from openpyxl.chart.label import DataLabelList
wb = load_workbook(filename="Survey.xlsx") 
ws = wb.active

for ws in wb.worksheets:        
    for row in ws.iter_rows(min_col=1, max_col=1):
        if row[0].value is None or row[0].value == "Reason":
            maxrowchart = (row[0].row - 1)
            break
    values = Reference(ws, min_col= 2, min_row=3, max_col = 2, max_row = maxrowchart)
    cats = Reference(ws, min_col=1, max_col=1, min_row=4, max_row= maxrowchart)

# Create object of BarChart class
    chart = BarChart()
    chart.height = 10
    chart.width = 15
    chart.add_data(values, titles_from_data=True)
    chart.set_categories(cats)
                # set the title of the chart
    chart.title = ws["A2"].value

                # the top-left corner of the chart
                # is anchored to cell F2 .
    chart.varyColors = "000F0FFF"
        
    for i in range(1,50):
        s=ws[f"A{i}"].value
        if s == 'Answer Choices': 
            ws.add_chart(chart,"H2")
wb.save("new.xlsx")

### Now, each multiple questionsheet has two charts

In [2]:
len(ws._charts)

2

### 2. Convert/Parse EXCEL to WORD "new.docx"
#### Note: this Word doc is not final output: it just parse all the contents from excel

(for now didn't work well by running it myself; instead i use the website below to get the word "new.docx")

In [46]:
#      https://products.aspose.com/cells/python-java/conversion/excel-to-word/

# Convert/Parse EXCEL to WORD

# import jpype     
# import asposecells     
# jpype.startJVM() 
# from asposecells.api import Workbook
# workbook = Workbook("new.xlsx")
# workbook.Save("new.docx")
# jpype.shutdownJVM()
	

### 3. get all the images in "new.docx" and save it to an already-exist folder "test"

In [5]:
# create empty folder "test"
import os
import shutil

dir = 'test'
if os.path.exists(dir):
    shutil.rmtree(dir)
os.makedirs(dir)

### EXTRACT ALL THE IMAGES IN "new.DOC
import docx2txt
#extract text 
text = docx2txt.process(r"new.docx")
#extract text and write images in Temporary Image directory
text = docx2txt.process(r"new.docx",r"test")



### note: in the folder we can see both one-color and multi-color chart for each question;
###       current idea is to only use "even number" image (e.g image 2, image 4, ...)

## NEXT: -figure out to put the image into final docx in correct order...   
##      idea: Dictionary, list to store the name of spreadsheets/questions who has multiple choices